In [9]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import sklearn.ensemble as ske
import numpy as np
import sklearn.metrics as skm


In [10]:
df = pd.read_csv("/Users/liamtabibzadeh/Documents/sml/sml-project-g20/data/train.csv", na_values='?')

df["difference_words_m_f"]=df.iloc[:,7]-df.iloc[:,0]

mean_difference=np.mean(df.difference_words_m_f)
def compute_epsilon(alpha,n,a,b):
    return np.sqrt((b-a)**2*np.log(2/alpha)/(2*n))

alpha = 0.05
n = len(df)
a = min(df.difference_words_m_f)
b = max(df.difference_words_m_f)
delta = compute_epsilon(alpha,n,a,b)
con=(mean_difference-delta,mean_difference+delta)
con

(887.0475434920338, 3568.1516865368403)

In [11]:

def year_sum(df):
    dictt={}
    for i in range(len(df)):
        current_year=df.loc[i,"Year"]
        if current_year in dictt:
            dictt[current_year]=dictt[current_year] + df.loc[i,"difference_words_m_f"]
        else:
            dictt[current_year]=df.loc[i,"difference_words_m_f"]
    return dictt
d=year_sum(df)
d=collections.OrderedDict(sorted(d.items()))
x=list(d.keys())
y=list(d.values())
#plt.scatter(x,y)


In [12]:

###CI for Gross for male dominating movies
alpha = 0.05
n = len(df.loc[ df["difference_words_m_f"]>0 ])
a = min(df.loc[ df["difference_words_m_f"]>0 ,"Gross"])
b = max(df.loc[ df["difference_words_m_f"]>0 ,"Gross"])
delta = compute_epsilon(alpha,n,a,b)
mean_gross_male=np.mean(df.loc[ df["difference_words_m_f"]>0 ,"Gross"])
confidence_I_male=(mean_gross_male-delta,mean_gross_male+delta)

###CI for Gross for female dominating movies
alpha = 0.05
n = len(df.loc[ df["difference_words_m_f"]<0 ])
a = min(df.loc[ df["difference_words_m_f"]<0 ,"Gross"])
b = max(df.loc[ df["difference_words_m_f"]<0 ,"Gross"])
delta = compute_epsilon(alpha,n,a,b)
mean_gross_female=np.mean(df.loc[ df["difference_words_m_f"]<0 ,"Gross"])
confidence_I_female=(mean_gross_female-delta,mean_gross_female+delta)

In [17]:
import sklearn.model_selection as skms
import numpy as np
import sklearn.tree as skt
import sklearn.linear_model as sklm
import random
import sklearn.discriminant_analysis as skda
random.seed(2)
logistic_object=sklm.LogisticRegression(max_iter=10000)
decisiontree_object=skt.DecisionTreeClassifier(max_depth=10,criterion="entropy")
randomforest_object=ske.RandomForestClassifier(n_estimators=100,criterion="entropy")
objects=[logistic_object,decisiontree_object,randomforest_object]
for x in objects:
    boosting_object=ske.AdaBoostClassifier(base_estimator=x,n_estimators=10)
    X=df.iloc[:,0:13]
    y=df.iloc[:,13]
    k_folds_object=skms.StratifiedKFold(n_splits=10,shuffle=True)

    acc_score = []
    for train_index , test_index in k_folds_object.split(X,y):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
        
        boosting_object.fit(X_train,y_train)
        pred_values = boosting_object.predict(X_test)
        
        acc = skm.accuracy_score(pred_values , y_test)
        acc_score.append(acc)
    avg_acc_score = np.mean(acc_score)
    print("Average accuracy for:",x,avg_acc_score)

Average accuracy for: LogisticRegression(max_iter=10000) 0.8527352501867064
Average accuracy for: DecisionTreeClassifier(criterion='entropy', max_depth=10) 0.8228715459297982
Average accuracy for: RandomForestClassifier(criterion='entropy') 0.845995145631068
